In [1]:
from collections import defaultdict
import re
import math
import pprint

def add_key(d, key, value):
    r = dict(d)
    r[key] = value
    return r

def tree():
    return defaultdict(tree)

def dicts(t, node_list): 
    return [add_key(node_list[k], 'node', dicts(t[k],node_list)) for k in t]

def get_level_content(s:str):
    s = re.split('(?<=\!)\!(?!\!)|^\!(?!\!)',s)
    return s[0].count('!'), s[1]

In [ ]:
str = '!(!!/!!)!!health > 0.0147!54: 8'
re.split(r'\(|\)|/', str)

In [ ]:
dd = defaultdict(int)
dd['d']=1
dd['w']=2
for di in dd:
    print(type(di))

In [ ]:
k = []
k.append(1)
print(k)
k.append(2)
print(k)
k.append(3)
print(k)
t = k.pop()
print(k)
k.append(t)
print(k)
print(k[-1])

In [2]:
def interpret_content(s, node):
    colon = re.search(r':(?= )', s)
    curr_name = ""
    if colon is None:
        t_content = s.split(' ')
        if len(t_content) == 3:
            return {'name':t_content[0], 'op':t_content[1], 'threshold':float(t_content[2]), 'node':node}
        else:
            print('格式错误: ', s)
            return None
    else:
        # 结果字段的解析
        t_result = s[colon.span()[0]+1:].strip().split(' ')
        curr_result = int(t_result[0])
        instance_num = re.split(r'\(|\)|/', t_result[1].strip())
        while '' in instance_num:
            instance_num.remove('')
        # 计算节点正确错误的数目
        if len(instance_num) == 2:
            right_num = float(instance_num[0])-float(instance_num[1])
            wrong_num = float(instance_num[1])
        elif len(instance_num) == 1:
            right_num = float(instance_num[0])
            wrong_num = 0
        t_content = s[:colon.span()[0]].strip().split(' ')
        # 找出节点0和1的数目
        zero_num = 0
        one_num = 0
        if curr_result == 0:
            zero_num = right_num
            one_num = wrong_num
        elif curr_result == 1:
            zero_num = wrong_num
            one_num = right_num
        # 得到节点内容
        if len(t_content) == 3:
            return {'name':t_content[0], 'op':t_content[1], 'threshold':float(t_content[2]), "result":curr_result, 'zeros':zero_num, 'ones':one_num, 'node':node}
        else:
            print('格式错误: ', s)
            return None

In [28]:
t = tree()
curr_node = t
node_num = 1
node_list = [{'node':t}]
travel_stack = [0]
last_level = -1
f = open('TreeMaps\\TreeMap-细分维度')
for s in f.readlines():
    curr_level, curr_content = get_level_content(s)
#     print(node_num, '\t', curr_level, '\t', travel_stack)
    if(curr_level > last_level):
        curr_node = curr_node[node_num]
    else:
        skip_level = last_level-curr_level+1
        for skip_step in range(skip_level):
            travel_stack.pop()
        curr_node = node_list[travel_stack[-1]]['node'][node_num]
    t_dict = interpret_content(curr_content, curr_node)
    if t_dict is not None:
        node_list.append(t_dict)
    else:
        break
    travel_stack.append(node_num)
    node_num += 1
    last_level = curr_level
res_d = dicts(t, node_list)

In [6]:
import json
pprint.pprint(res_d)
# jsObj = json.dumps(res_d)  
  
# fileObject = open('jsonFileChi.json', 'w')  
# fileObject.write(jsObj)  
# fileObject.close()  

[{'name': 'bio',
  'node': [{'name': 'bio',
            'node': [{'name': 'percept',
                      'node': [{'name': 'TenseM',
                                'node': [],
                                'ones': 0,
                                'op': '<=',
                                'result': 0,
                                'threshold': 0.007092,
                                'zeros': 196.0},
                               {'name': 'TenseM',
                                'node': [{'name': 'pronoun',
                                          'node': [{'name': 'affect',
                                                    'node': [{'name': 'percept',
                                                              'node': [{'name': 'TenseM',
                                                                        'node': [{'name': 'social',
                                                                                  'node': [],
                                       

                                          'threshold': 0.152249}],
                                'op': '<=',
                                'threshold': 0.015089},
                               {'name': 'percept',
                                'node': [{'name': 'TenseM',
                                          'node': [],
                                          'ones': 3.0,
                                          'op': '<=',
                                          'result': 0,
                                          'threshold': 0.028145,
                                          'zeros': 62.0},
                                         {'name': 'TenseM',
                                          'node': [{'name': 'pronoun',
                                                    'node': [{'name': 'TenseM',
                                                              'node': [],
                                                              'ones': 3.0,
                     

In [ ]:
if res_d[0]['node'][0]['node'] == []:
    print('ok')

In [7]:
def cal_zero_one(t):
    if t['node'] == []:
        return t['zeros'], t['ones']
    zero_cnt = 0
    one_cnt = 0
    for child in t['node']:
        zero_t, one_t = cal_zero_one(child)
        zero_cnt += zero_t
        one_cnt += one_t
    return zero_cnt, one_cnt

In [8]:
def cal_right_wrong(t):
    zero_cnt, one_cnt = cal_zero_one(t)
    if zero_cnt >= one_cnt:
        return zero_cnt, one_cnt
    else:
        return one_cnt, zero_cnt

In [9]:
def cal_entropy_unit(ratio):
    if ratio == 0 or ratio == 1:
        return 0
    return -ratio*math.log(ratio, 2)

In [10]:
def cal_entropy(node):
    r, w = cal_right_wrong(node)
    r_ratio = r/(r+w)
    w_ratio = w/(r+w)
    
    return cal_entropy_unit(r_ratio)+cal_entropy_unit(w_ratio)

In [11]:
def cal_gain_ratio(node):
    if node['node'] == []:
        return 0
    fore_entropy = cal_entropy(node)
    r_num, w_num = cal_right_wrong(node)
    instance_num = r_num + w_num
    late_entropy = 0
    split_info = 0
    for child in node['node']:
        cr_num, cw_num = cal_right_wrong(child)
        c_instance_num = cr_num + cw_num
        c_entropy = cal_entropy(child)
        late_entropy += c_instance_num/instance_num*c_entropy
        c_split_ratio = c_instance_num/instance_num
        split_info += cal_entropy_unit(c_split_ratio)
    return (fore_entropy-late_entropy)/split_info

In [ ]:
(cal_zero_one(whole_tree['node'][0]),
cal_zero_one(whole_tree['node'][1]))

In [ ]:
print(cal_gain_ratio(res_d[0]['node'][1]))
pprint.pprint(res_d[0]['node'][1])

In [12]:
def add_gain_ratio(node):
    node['gain_ratio'] = cal_gain_ratio(node)
    for child in node['node']:
        add_gain_ratio(child)

In [13]:
def add_entropy(node):
    node['entropy'] = cal_entropy(node)
    for child in node['node']:
        add_entropy(child)

In [14]:
def add_zero_one(node):
    if node['node'] ==[]:
        return
    zeros, ones= cal_zero_one(node)
    node['zeros'] = zeros
    node['ones'] = ones
    for child in node['node']:
        add_zero_one(child)

In [29]:
whole_tree = {'node':[res_d[0], res_d[1]]} 
add_gain_ratio(whole_tree)
add_entropy(whole_tree)
add_zero_one(whole_tree)

In [30]:
pprint.pprint(whole_tree)

{'entropy': 0.7273571651393319,
 'gain_ratio': 0.13389624375480094,
 'node': [{'entropy': 0.9491337436654711,
           'gain_ratio': 0.31975959724764513,
           'name': 'health',
           'node': [{'entropy': 0.1571952258581087,
                     'gain_ratio': 0.16407183052534027,
                     'name': 'PresentM',
                     'node': [{'entropy': 0.10450184951478625,
                               'gain_ratio': 0.08438508593811365,
                               'name': 'PastM',
                               'node': [{'entropy': 0.06783960355883464,
                                         'gain_ratio': 0.02085760364503276,
                                         'name': 'shehe',
                                         'node': [{'entropy': 0,
                                                   'gain_ratio': 0,
                                                   'name': 'ProgM',
                                                   'node': [],
                  

                                                                                                                                                       'node': [{'entropy': 0,
                                                                                                                                                                 'gain_ratio': 0,
                                                                                                                                                                 'name': 'family',
                                                                                                                                                                 'node': [],
                                                                                                                                                                 'ones': 3.0,
                                                                                                                         

                                                            {'entropy': 0.2761954276479391,
                                                             'gain_ratio': 0.08214469579408239,
                                                             'name': 'tentat',
                                                             'node': [{'entropy': 0,
                                                                       'gain_ratio': 0,
                                                                       'name': 'incl',
                                                                       'node': [],
                                                                       'ones': 0,
                                                                       'op': '<=',
                                                                       'result': 0,
                                                                       'threshold': 0.045948,
                                               

                                                   'ones': 29.0,
                                                   'op': '>',
                                                   'threshold': 0.007344,
                                                   'zeros': 35.0}],
                                         'ones': 32.0,
                                         'op': '<=',
                                         'threshold': 0.007813,
                                         'zeros': 85.0},
                                        {'entropy': 0.29333653421235084,
                                         'gain_ratio': 0.03581561155209551,
                                         'name': 'body',
                                         'node': [{'entropy': 0.5334533565308074,
                                                   'gain_ratio': 0.24736558225007327,
                                                   'name': 'we',
                                                   'node': [{'e

In [31]:
import json
jsObj = json.dumps(whole_tree)  
fileObject = open('jsonTree-细分维度.json', 'w')  
fileObject.write(jsObj)  
fileObject.close()  